In [2]:
import cv2
import numpy as np

In [3]:
corner_track_params = dict(maxCorners = 10, qualityLevel = 0.3, minDistance = 7, blockSize = 7) # corner_track_params ile köşeleri belirliyoruz

In [4]:
lk_params = dict(winSize = (200,200), maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)) # winSize ile hareket eden nesnenin boyutunu /hassasiyetini belirliyoruz maxLevel ile de hareket eden nesnenin çözünürlüğü belirliyoruz criteria ile de hareket eden nesnenin hassasiyetini belirliyoruz , 10 ile 0.03 değerleri arasında bir hassasiyet belirliyoruz 

In [5]:
cap= cv2.VideoCapture(0)

ret, prev_frame = cap.read()

prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

prevPts = cv2.goodFeaturesToTrack(prev_gray, mask = None, **corner_track_params) # prevPts ile de prev_gray içindeki köşeleri belirliyoruz
# **corner_track_params ile de corner_track_params içindeki değerleri prevPts içine atıyoruz dictionary yapısını kullandığımız içi ** kullandık

mask = np.zeros_like(prev_frame) # mask ile de prev_frame içindeki değerleri 0 yapıyoruz. çünkü prev_frame içindeki değerleri 0 yapmazsak hareket eden nesnenin etrafına bir kare çiziyor . bu kareyi çizmemek için prev_frame içindeki değerleri 0 yapıyoruz . önceki karenin eşleştiği kareyi buluyoruz

while True:
    ret, frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, prevPts, None, **lk_params) # nextPts ile de prev_gray ile frame_gray arasındaki farkı buluyoruz. status ile de nextPts içindeki değerleri buluyoruz. err ile de nextPts içindeki değerleri buluyoruz

    good_new = nextPts[status == 1] # status içindeki değerleri 1 olanları good_new içine atıyoruz

    good_prev = prevPts[status == 1] # status içindeki değerleri 1 olanları good_prev içine atıyoruz

    for i, (new, prev) in enumerate(zip(good_new, good_prev)): # good_new ve good_prev içindeki değerleri zip ile birleştiriyoruz
        x_new, y_new = new.ravel() # new içindeki değerleri ravel ile düzleştiriyoruz
        x_prev, y_prev = prev.ravel() # prev içindeki değerleri ravel ile düzleştiriyoruz

        mask = cv2.line(mask, (x_new, y_new), (x_prev, y_prev), (0, 255, 0), 3) # mask içindeki değerleri çiziyoruz

        frame = cv2.circle(frame, (x_new, y_new), 8, (0, 0, 255), -1) # frame içindeki değerleri çiziyoruz

    img = cv2.add(frame, mask) # frame ve mask içindeki değerleri topluyoruz birleştiririyoruz

    cv2.imshow("tracking", img)

    k = cv2.waitKey(30) & 0xFF
    if k == 27:
        break

    prev_gray = frame_gray.copy()
    prevPts = good_new.reshape(-1, 1, 2) # good_new içindeki değerleri reshape ile düzleştiriyoruz ve prevPts içine atıyoruz 

cv2.destroyAllWindows()
cap.release()


error: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'line'
> Overload resolution failed:
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type
